In [43]:
!pip install transformers==3
!pip install torchinfo

  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached tokenizers-0.8.0rc4.tar.gz (96 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [44]:
import numpy as np
import pandas as pd
import re
import torch
import random
import torch.nn as nn
import transformers
import matplotlib.pyplot as plt
# specify GPU
# device = torch.device("cuda")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
df = pd.read_csv("/content/updated_complaints.csv")
df.head()

,customer_id,complaint_id,time_of_complaint,date_of_complaint,questions,response_text,Intent,Unnamed: 7,Unnamed: 8
0,10001,2001,10:30 AM,2022-01-01,Why is the product delayed?,We apologize for the delay. There was a high d...,Product,NaN,NaN
1,10002,2002,10:35 AM,2022-01-02,Can you provide more details about product XYZ?,Certainly! Product XYZ is a high-quality elect...,Product,NaN,NaN
2,10003,2003,10:40 AM,2022-01-03,The product arrived damaged. What should I do?,We're sorry to hear that. Please provide your ...,Product,NaN,NaN
3,10004,2004,10:45 AM,2022-01-04,How can I track my order?,You can track your order by logging into your ...,Product,NaN,NaN
4,10005,2005,10:50 AM,2022-01-05,Is there a warranty on this product?,Yes our products come with a standard one-year...,Product,NaN,NaN


In [46]:
df['Intent'].value_counts()

Product              40
Positive Feedback    40
Negative Feedback    40
Product Inquiry      20
Greetings            20
Name: Intent, dtype: int64

In [47]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['Intent'])
df['label'].value_counts(normalize = True)

3    0.250
2    0.250
1    0.250
4    0.125
0    0.125
Name: label, dtype: float64

In [48]:
train_text, train_labels = df["questions"], df["label"]
train_labels.unique()

array([3, 2, 1, 4, 0])

In [49]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [50]:
# tokenize and encode sequences in the training set
max_seq_len = 8
tokens_train = tokenizer(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [51]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())
train_y

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [53]:
class BERT_Arch(nn.Module):
   def __init__(self, bert):
       super(BERT_Arch, self).__init__()
       self.bert = bert

       self.dropout = nn.Dropout(0.2)

       self.relu =  nn.ReLU()
       self.fc1 = nn.Linear(768,512)
       self.fc2 = nn.Linear(512,256)
       self.fc3 = nn.Linear(256,5)
       self.softmax = nn.LogSoftmax(dim=1)
   def forward(self, sent_id, mask):
      cls_hs = self.bert(sent_id, attention_mask=mask)[0][:,0]

      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)

      x = self.fc2(x)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc3(x)

      x = self.softmax(x)
      return x

In [54]:
for param in bert.parameters():
      param.requires_grad = False
model = BERT_Arch(bert)
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                  Param #
BERT_Arch                                               --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              (23,440,896)
│    │    └─Embedding: 3-2                              (393,216)
│    │    └─LayerNorm: 3-3                              (1,536)
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             (42,527,232)
├─Dropout: 1-2                                          --
├─ReLU: 1-3                                             --
├─Linear: 1-4                                           393,728
├─Linear: 1-5                                           131,328
├─Linear: 1-6                                           1,285
├─LogSoftmax: 1-7                                       --
Total 

In [55]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [56]:
from sklearn.utils.class_weight import compute_class_weight

class_labels = np.unique(train_labels)
class_wts = compute_class_weight(class_weight ='balanced', classes=class_labels, y=np.array(train_labels))

class_weights = dict(zip(class_labels, class_wts))

In [57]:
print(class_wts)

[1.6 0.8 0.8 0.8 1.6]


In [58]:
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)
cross_entropy = nn.NLLLoss(weight=weights)

In [59]:
from torch.optim import lr_scheduler
train_losses=[]
epochs = 200
lr_sch = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)


In [60]:
def train():

  model.train()
  total_loss = 0

  total_preds=[]

  for step,batch in enumerate(train_dataloader):

    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step,    len(train_dataloader)))
    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    preds = model(sent_id, mask)
    loss = cross_entropy(preds, labels)
    total_loss = total_loss + loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    optimizer.zero_grad()

    preds=preds.detach().cpu().numpy()
    total_preds.append(preds)
  avg_loss = total_loss / len(train_dataloader)

  total_preds  = np.concatenate(total_preds, axis=0)
  # torch.save(model, 'classify_intent.pt')
  return avg_loss, total_preds

In [61]:
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    model.to(device)
    train_loss, _ = train()

    train_losses.append(train_loss)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
print(f'\nTraining Loss: {train_loss:.3f}')


 Epoch 1 / 200

 Epoch 2 / 200

 Epoch 3 / 200

 Epoch 4 / 200

 Epoch 5 / 200

 Epoch 6 / 200

 Epoch 7 / 200

 Epoch 8 / 200

 Epoch 9 / 200

 Epoch 10 / 200

 Epoch 11 / 200

 Epoch 12 / 200

 Epoch 13 / 200

 Epoch 14 / 200

 Epoch 15 / 200

 Epoch 16 / 200

 Epoch 17 / 200

 Epoch 18 / 200

 Epoch 19 / 200

 Epoch 20 / 200

 Epoch 21 / 200

 Epoch 22 / 200

 Epoch 23 / 200

 Epoch 24 / 200

 Epoch 25 / 200

 Epoch 26 / 200

 Epoch 27 / 200

 Epoch 28 / 200

 Epoch 29 / 200

 Epoch 30 / 200

 Epoch 31 / 200

 Epoch 32 / 200

 Epoch 33 / 200

 Epoch 34 / 200

 Epoch 35 / 200

 Epoch 36 / 200

 Epoch 37 / 200

 Epoch 38 / 200

 Epoch 39 / 200

 Epoch 40 / 200

 Epoch 41 / 200

 Epoch 42 / 200

 Epoch 43 / 200

 Epoch 44 / 200

 Epoch 45 / 200

 Epoch 46 / 200

 Epoch 47 / 200

 Epoch 48 / 200

 Epoch 49 / 200

 Epoch 50 / 200

 Epoch 51 / 200

 Epoch 52 / 200

 Epoch 53 / 200

 Epoch 54 / 200

 Epoch 55 / 200

 Epoch 56 / 200

 Epoch 57 / 200

 Epoch 58 / 200

 Epoch 59 / 200

 Epoc

In [62]:
def get_prediction(str):
 str = re.sub(r'[^a-zA-Z ]+', '', str)
 test_text = [str]
 model.eval()

 tokens_test_data = tokenizer(
 test_text,
 max_length = max_seq_len,
 pad_to_max_length=True,
 truncation=True,
 return_token_type_ids=False
 )
 test_seq = torch.tensor(tokens_test_data['input_ids'])
 test_mask = torch.tensor(tokens_test_data['attention_mask'])

 preds = None
 with torch.no_grad():
   preds = model(test_seq.to(device), test_mask.to(device))
 preds = preds.detach().cpu().numpy()
 preds = np.argmax(preds, axis = 1)
 print("Intent Identified: ", le.inverse_transform(preds)[0])
 return le.inverse_transform(preds)[0]

In [66]:
import json
import pandas as pd

In [65]:
intent_list = [{"tag" : "Product",
                "responses" : []},
               {"tag" : "Positive Feedback",
                "responses" : []},
               {"tag" : "Negative Feedback",
                "responses" : []},
               {"tag" : "Product Inquiry",
                "responses" : []},
               {"tag" : "Greetings",
                "responses" : []}]

In [69]:
for index, row in df.iterrows():
    for index, item in enumerate(intent_list):
        if item["tag"] == "Product":
            intent_list[index]['responses'].append(row['response_text'])
        elif item["tag"] == "Positive Feedback":
            intent_list[index]['responses'].append(row['response_text'])
        elif item["tag"] == "Negative Feedback":
            intent_list[index]['responses'].append(row['response_text'])
        elif item["tag"] == "Product Inquiry":
            intent_list[index]['responses'].append(row['response_text'])
        elif item["tag"] == "Greetings":
            intent_list[index]['responses'].append(row['response_text'])

In [70]:
data_dict = {"intents" : intent_list}

In [71]:
def get_response(message):
  intent = get_prediction(message)
  for i in data_dict['intents']:
    if i["tag"] == intent:
      result = random.choice(i["responses"])
      break
  print(f"Response : {result}")
  return "Intent: "+ intent + '\n' + "Response: " + result

In [84]:
get_response("My product is not delivered")

Intent Identified:  Product
Response : To track the delivery status of your order for Product K you can use the order tracking feature on our website. Alternatively you can provide me with your order number and I'll check it for you.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


"Intent: Product\nResponse: To track the delivery status of your order for Product K you can use the order tracking feature on our website. Alternatively you can provide me with your order number and I'll check it for you."

In [93]:
get_response("My product is damaged")

Intent Identified:  Product
Response : We're sorry for any difficulty reaching us. Please provide more details about your attempts to contact us and we'll investigate the issue. We appreciate your patience and understanding.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


"Intent: Product\nResponse: We're sorry for any difficulty reaching us. Please provide more details about your attempts to contact us and we'll investigate the issue. We appreciate your patience and understanding."

In [89]:
get_response("My product is not upto what I expected")

Intent Identified:  Product
Response : Hey! Absolutely I can help with that. If you tell me the product you purchased I'll suggest similar items or alternatives that you might find interesting.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


"Intent: Product\nResponse: Hey! Absolutely I can help with that. If you tell me the product you purchased I'll suggest similar items or alternatives that you might find interesting."